In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [0]:
%scala
val appId = dbutils.secrets.get(scope = "", key = "clientid")
val tenantId = dbutils.secrets.get(scope = "", key = "tenantid")

appId: String = [REDACTED]
tenantId: String = [REDACTED]

In [0]:
%scala
spark.conf.set(s"spark.abfs.custom.oauth2.client.id", appId)
spark.conf.set(s"spark.abfs.custom.oauth2.tenant.id",  tenantId)
spark.conf.set(s"fs.azure.account.auth.type", "Custom")
spark.conf.set(s"fs.azure.account.oauth.provider.type","")

In [0]:
df_GoogleData = spark.read.format("csv").option("header","true").option("inferSchema", "true").load("Path/_a_AmitKumar/googleplaystore.csv")

In [0]:
df_GoogleData.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



In [0]:
df_GoogleData = df_GoogleData.drop("Size","Content Rating","Last Updated", "Android Ver","Current Ver")

In [0]:
df_GoogleData = df_GoogleData.withColumn("Reviews",col("Reviews").cast("int"))\
  .withColumn("Installs",regexp_replace(col("Installs"), "[^0-9]", ""))\
  .withColumn("Installs",col("Installs").cast("int"))\
  .withColumn("Price",regexp_replace(col("Price"), "[$]", ""))\
  .withColumn("Price",col("Price").cast("float"))

In [0]:
df_GoogleData.createOrReplaceTempView("GoogleData")

In [0]:
%sql
SELECT
  App,
  SUM(Reviews) AS TotalReviews
FROM
  GoogleData
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

App,TotalReviews
Instagram,266241989
WhatsApp Messenger,207348304
Clash of Clans,179558781
Messenger – Text and Video Chat for Free,169932272
Subway Surfers,166331958
Candy Crush Saga,156993136
Facebook,156286514
8 Ball Pool,99386198
Clash Royale,92530298
Snapchat,68045010


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
  App,
  SUM(Installs) AS TotalInstalls
FROM
  GoogleData
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

App,TotalInstalls
Subway Surfers,6000000000
Hangouts,4000000000
Google Photos,4000000000
Google News,4000000000
Google Drive,4000000000
Instagram,4000000000
Candy Crush Saga,3500000000
WhatsApp Messenger,3000000000
Messenger – Text and Video Chat for Free,3000000000
Google Chrome: Fast & Secure,3000000000


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
  Category,
  SUM(Installs) AS TotalInstalls
FROM
  GoogleData
WHERE Category NOT IN ("1.9")
GROUP BY 1
ORDER BY 2 DESC

Category,TotalInstalls
GAME,35086024415
COMMUNICATION,32647276251
PRODUCTIVITY,14176091369
SOCIAL,14069867902
TOOLS,11452771915
FAMILY,10258263505
PHOTOGRAPHY,10088247655
NEWS_AND_MAGAZINES,7496317760
TRAVEL_AND_LOCAL,6868887146
VIDEO_PLAYERS,6222002720


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT
  App,
  ROUND(SUM(Price),2) AS TotalPrice
FROM
  GoogleData
WHERE Type = "Paid"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

App,TotalPrice
I'm Rich - Trump Edition,400.0
I'm Rich/Eu sou Rico/أنا غني/我很有錢,399.99
I Am Rich Pro,399.99
I Am Rich Premium,399.99
most expensive app (H),399.99
I AM RICH PRO PLUS,399.99
I am Rich,399.99
I am Rich!,399.99
I am Rich Plus,399.99
I am rich(premium),399.99
